# Supervisor - Worker Controlled Flow for Gen Pod AI Backend


## Import Modules Needed for this Project

In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

from langchain_core.messages import AIMessage
from langchain_core.messages import ToolMessage

from langchain_core.messages.tool import ToolCall

from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import ToolExecutor
from langgraph.prebuilt import ToolInvocation

from langgraph.graph import END
from langgraph.graph import StateGraph

from langgraph.checkpoint.sqlite import SqliteSaver

# from langchain_core.agents import AgentAction
# from langchain_core.agents import AgentFinish

from langchain.schema import Document

from langchain.tools import tool

from langchain_openai import ChatOpenAI

from typing import List
from typing import Dict
from typing import Union
from typing import Literal
from typing import ClassVar
from typing import Sequence
from typing import TypedDict
from typing import Annotated

from pydantic import Field
from pydantic import BaseModel
from pydantic import ValidationError

from typing_extensions import List
from typing_extensions import TypedDict

from dotenv import load_dotenv

from enum import Enum

from IPython.display import Image
from IPython.display import display

import pprint as pp

import os
import re
import ast
import json
import requests
import operator
import subprocess

ModuleNotFoundError: No module named 'langchain_core'

Let us import some basic libraries and load dotenv file, make sure it contains necessary API keys.

In [ ]:
load_dotenv()

### Constants for the project       

In [ ]:
# node names used in graph
class GraphNodes(Enum):
    ARCHITECT: str = "ARCHITECT"

    CODER: str = "CODER"

    EXECUTE_COMMAND: str = "EXECUTE_COMMAND"
    CREATE_GIT_REPO: str = "CREATE_GIT_REPO"
    VERIFY_FILE_CONTENT: str = "VERIFY_FILE_CONTENT"
    CHECK_FILES_CREATED: str = "CHECK_FILES_CREATED"
    DOWNLOAD_LICENSE_FILE: str = "DOWNLOAD_LICENSE_FILE"
    WRITE_GENERATED_CODE_TO_FILE: str = "WRITE_GENERATED_CODE_TO_FILE"

    START: str = "__START__"
    END: str = END

    CALL_TOOL: str = "CALL_TOOL"                     
    
    NONE: str = ""

    def __str__(self):
        return self.value

# Task states decided by llm
class TaskState(Enum):
    NEW: str = "NEW"
    AWAITING: str = "AWAITING"
    HALT: str = "HALT"
    PENDING: str = "PENDING"
    COMPLETED: str = "COMPLETED"
    INCOMPLETE: str = "INCOMPLETE"
    DONE: str = "DONE"

    def __str__(self):
        return self.value

# roles that gonna be used while adding messages to graph state
class ChatRoles(Enum):
    AI: str = "assistant"
    TOOL: str = "tool"
    USER: str = "user"
    SYSTEM: str = "system"

    def __str__(self):
        return self.value

# whitelisted commands for the llm that can be used to complete the task
ALLOWED_COMMANDS = ['mkdir', 'docker', 'python', 'python3', 'pip', 'virtualenv', 'mv', 'pytest', 'touch', 'cat', 'ls', 'curl'] #uvicorn

# Update this path to your local directory where you want to create the project at.
PROJECT_PATH = os.getenv('PROJECT_PATH','/This/Is/The/Path/To/Create/Your/Project/')
print("PROJECT_PATH: ", PROJECT_PATH)

#### Enums value behaviour

we need its value so we use second method 

In [ ]:
ChatRoles.AI

In [ ]:
ChatRoles.AI.value

### Variables and instances      for the projects

In [ ]:
## Loading LLM
# We will load OpenAI GPT-4o LLM to assist our agents.
llm = ChatOpenAI(model="gpt-4o-2024-05-13", temperature=0, max_retries=5, streaming=True, seed=4000)
code_llm = ChatOpenAI(model="gpt-4-turbo-2024-04-09",temperature=0.3, max_retries=5, streaming=True, seed=4000)

# tools that coder agent can access
coder_tools = ["write_generated_code_to_file", "create_git_repo", "execute_command", "download_license_file"]

## Let us create some Utility function
These function can help later on to read the input files as a json string and create agents that can be later used as nodes in the graph.

In [ ]:
def read_input_json(file_path) -> str:
    """Reads JSON data from a file and returns it as a string.

    Args:
        file_path: The path to the JSON file.

    Returns:
        A string representation of the JSON data.
    """
    with open(file_path, 'r') as user_input_file:
        data = json.load(user_input_file)
    
    user_input = json.dumps(data)
    license_txt = data["LICENSE_TEXT"]

    return user_input, license_txt

## Define tools to be used by Agents
These tools are custom fuctions that will also go as nodes in the graph and will be called by the agent to take some action.

In [ ]:
@tool
def download_license_file(
        url: Annotated[str, "LICENSE_URL from where it has to be downloaded."],
        file_path: Annotated[str, "Absolute path where the License.md should be written can handle directory create if does not exist."]
 ) -> str:
    """
    Downloads a license file from a given URL and saves it locally.

    Args:
    url (str): The URL of the lic
    ense file.
    file_path (str): Absolute path where the generated code should be written can handle directory create if does not exist.
    
    Returns:
    str: The local path where the file was saved.
    """
    import pprint as pp
    response = requests.get(url)
    # print(response.content)
    # response.raise_for_status()  # Raise exception if the request failed
    try:
        os.makedirs(os.path.dirname(file_path), exist_ok=True)

        with open(file_path, 'wb') as file:
            file.write(response.content)
        return f"Successfully wrote the License to {file_path}"

    except:
        return f"failed to write the License to {file_path}"

@tool
def write_generated_code_to_file(
    generated_code: Annotated[str, "The code generated by the agent."],
    file_path: Annotated[str, "Absolute path where the generated code should be written can handle directory create if does not exist."]
):
    """
    Writes the provided generated code to the specified file within the project structure.

    Args:
        generated_code (str): The code generated by the agent.
        file_path (str): The path where the generated code should be written.
    """
    try:

        # Ensure the directory exists before writing the file
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        
        with open(file_path, 'w') as file:
            file.write(generated_code)
        
        return {'project_files':file_path}, f"Successfully wrote the generated code to: {file_path}"
    except BaseException as e:
        return f"Failed to write generated code. Error: {repr(e)}"
    
@tool
def create_git_repo(
    project_name: Annotated[str, "Name of the new Git repository that should be created."],
    repo_path: Annotated[str,"Path where the repository is created."]
):
    """
    Creates a new Git repository at the specified path.

    Args:
        project_name (str): Name of the new Git repository that should be created.
        PROJECT_PATH (str): Path where the new Git repository will be created.

    Returns:
        A dictionary containing the path of the newly created Git repository or an error message.
    """
    try:
        repo_path = os.path.join(PROJECT_PATH, project_name)
        
        # Ensure the directory exists before initializing the Git repository
        os.makedirs(repo_path, exist_ok=True)
        
        subprocess.check_output(['git', 'init'], cwd=repo_path)
        
        return {'repo_path': repo_path}, f"Git repository created successfully: {repo_path}"
    except Exception as e:
        return f"Failed to create a new Git repository. Error: {repr(e)}"

@tool
def execute_command(
    command: Annotated[str, "The complete set of commands to be executed on the local machine in order."],
    repo_path: Annotated[str,"Path where the repository is created."]
) -> str:
    """
    Executes a command on the local machine. This function is only allowed to use the following commands:

    'mkdir', 'docker', 'python', 'python3', 'pip', 'virtualenv', 'mv', 'pytest',
    'touch', 'cat', 'ls', 'curl'

    Args:
        command (str): The complete set of commands to be executed on the local machine in order.
        repo_path(str): absolute path where the command has to run.
    """
    # Split the command into parts
    parts = command.split()
    
    # Check if the command is in the whitelist
    if parts[0] not in ALLOWED_COMMANDS:
        return f"Command '{parts[0]}' is not allowed."
    
    try:
        # Execute the command
        # full_path = os.path.join(PROJECT_PATH,repo_path)
        additional_command = f"cd {repo_path} && "
        updated_command = additional_command + command
        result = subprocess.check_output(updated_command, shell=True)
        
        return f"Command executed successfully. Output: {result}"
    except BaseException as e:
        return f"Failed to execute command. Error: {repr(e)}"
    
@tool
def check_files_created(
    files: Annotated[List[str], "The list of files that should be present in the project repository."],
    repo_path: Annotated[str, "Absolute Path where the repository is created."]
):
    """
    Checks if all the specified files within a folder structure are created or not.

    Args:
        files (List[str]): The list of files to check.
    """
    missing_files = []
    
    # Check each file
    for file in files:
        full_file_path = os.path.join(repo_path, file)
        
        # Directly check if the file exists without executing shell commands
        if not os.path.exists(full_file_path):
            missing_files.append(file)
    
    if missing_files:
        return {"missing_files":missing_files}, f"missing these files: {missing_files}"
    else:
        return "All files are present."
    
@tool
def verify_file_content(
    files: Annotated[List[str], "The list of files that should be present in the project repository."],
    repo_path: Annotated[str, "Absolute Path where the repository is created."]
):
    """
    Checks if all the specified files within a folder structure are empty or not.

    Args:
        files (List[str]): The list of files to check.
    """
    
    empty_files = []
    
    # Check each file
    for file in files:
        
        full_file_path = os.path.join(repo_path, file)
        
        # Check if the file exists and is empty
        if os.path.exists(full_file_path) and os.path.getsize(full_file_path) == 0:
            empty_files.append(file)
    
    if empty_files:
        return f"These files are empty: {empty_files}"
    else:
        return "All files are not empty."


coder_tools_list = [write_generated_code_to_file, create_git_repo, execute_command, download_license_file]


## Let's Define a New State that will maintain all the statespace needed to run the architect coder flow smoothly.

In [ ]:
class GraphState(TypedDict):
    """
    
    Represents the state of our graph.

    Attributes:
        messages : With user question, error messages, reasoning
        generation : Code solution
        iterations : Number of tries
    """

    error: Annotated[bool, "tells if previous step has encountered an error."] = Field(default=False)
    generation: Annotated[str, Field(default="")] # TODO: Need to proper description for field
    iterations: Annotated[int, Field(default=0, description="number of steps the current flow has took till now")]
    messages: Annotated[list, Field(default=[], description="all messages that were generated during the flow")]
    # when ever coder need to use tools. it assigns one tool call from pending tool calls to here.
    curr_tool_call: Annotated[dict, Field(default={}, description="holds the details required for the current tool call.")]
    curr_task_status: Annotated[str, Field(default=TaskState.NEW.value, description="status of the current task assigned to coder.")]
    current_step: Annotated[str, Field(default="", description="current step coder tyring to finish")]

    # Architect controlled/updated fields
    project_name: Annotated[str, Field(default="", description="Project name that the user has assigned to work on")]
    requirements_overview: Annotated[str, Field(default="", description="A comprehensive summary of the project’s requirements, outlining the necessary functionalities and features.")]
    tasks: Annotated[list[str], Field(default=[], description="list of tasks with detailed requirements into independent task with as much context as possible that are crucial to follow during completion")]
    project_folder_structure: Annotated[str, Field(default="", description="Project folder structure to follow.")]
    current_task: Annotated[str, Field(default="", description="Task to do with all the functional and non functional details related to that task")]
    
    # Coder controlled/updated fields
    file_path: Annotated[str, Field(description="Depending on the project structure where should the code be written to"), operator.add]
    code: Annotated[str, Field(description="Fully complete, well documented code, with all the naming standards followed that is needed to complete the task."), operator.add]
    # needs to be updated after coder llm is called.
    coder_steps: Annotated[list, Field(default=[], description="Steps needed to complete coder task.")]
    coder_response: Annotated[str, Field(default="", description="Parsed Coder response after architect assigning the task to coder")]
    # if there are calls to different tools at the same time then they are stored here to schedule their execution.
    pending_tool_calls: Annotated[list, Field(default=[], description="list of tool calls requested by coder to complete tasks")]
    license_text: Annotated[str, Field(default="", description="A Licensing text from the user input that needs to be prefixed to each code.")]
    files_to_create: Annotated[str, "files need to be written to local."]
    # controlled/updated fields all nodes
    project_status: Annotated[str, Field(default=TaskState.NEW.value, description="current status of the project.", )]
    call_next: Annotated[str, Field(default=GraphNodes.NONE.value, description="Whom to call next")]
    max_retry: int

def add_message(state, message: tuple[str, str]) -> GraphState:
    """
    Adds a single message to the the messages

    message: tuple[str, str]

    Ex: add_message(state, ('user', 'single message'))
    """

    state['messages'] += [message]
    return state

def add_messages(state, messages: list[tuple[str, str]]) -> GraphState:
    """
    Adds a list of messages to the messages

    messages: list[tuple[str, str]]

    Ex: add_messages(state, [('user', 'message 1'), ('ai', 'message 2)])
    """
    state['messages'] += messages

    return state

def get_messages_for_prompt(state) -> list[tuple[str, str]]:
    """
    Returns last 5 messages from messages field.

    Ex: messages = get_messages_for_prompt(state)
    """

    return state['messages']

def get_last_message(state) -> tuple[str, str]:
    """
    Fetches the last message from messages field.
    
    Ex: last_message = get_last_message(state)
    """

    return state['messages'][-1]

def toggle_error(state) -> GraphState:
    """
    toggle error field. true -> false and flase -> true

    Ex: state.toggle_error()
    """

    state['error'] = not state['error']
    
    return state


def next_major_task(state) -> str:
    """
    returns 
        None when list is empty.
        The next available task when non empty.

    Ex: next_task = next_major_task(state)
    """

    if len(state['tasks']) == 0:
        return None
    
    return state['tasks'].pop(0)

def next_coder_step(state) -> str:
    """
    returns 
        None when list is empty.
        The next available step when non empty.

    Ex: next_step = state.next_coder_step()
    """

    if len(state['coder_steps']) == 0:
        return None
    
    return state['coder_steps'].pop(0)

def next_pending_tool_call(state) -> dict:
    """
    returns 
        None when list is empty.
        The next available tool call on pending_tool_call field.

    Ex: tool_call = state.next_pending_tool_call()        
    """

    if len(state['pending_tool_calls']) == 0:
        return None
    
    return state['pending_tool_calls'].pop(0)

def get_project_state(state) -> str:
    """
    returns current task status.

    Ex: task_status = state.get_project_state()
    """

    return state['project_status']

def get_next(state) -> str:
    """
    returns next node to be called.
    
    Ex: next = state.get_next()
    """

    return state['call_next']

def set_curr_tool_call_from_pending_tool_calls(state) -> GraphState:
    """
    updates the curr_tool_call from the pending tool calls.

    Ex: state.set_curr_tool_call_from_pending_tool_calls()
    """

    tool_call = next_pending_tool_call(state)
    if tool_call is None:
        state['curr_tool_call'] = {}
    else:
        state['curr_tool_call'] = tool_call

    return state

def get_curr_tool_call(state) -> dict:
    """
    returns the current tool call which need to be processed.

    Ex: tool_call = get_curr_tool_call(state)
    """

    return state['curr_tool_call']



## Let's us define a Data model that we can use to update the call's in the graph plus also to define a schema of response from the llm's

In [ ]:
# Pydantic class used by architect chain for structuring output
class RequirementsDoc(BaseModel):
    """Requirements Document output"""

    project_name: str = Field(description="Project name that the user has assigned you to work on", required=True)
    well_documented: str = Field(description="Well built requirements document from the user input", required=True)
    tasks: str = Field(description="Spilt the detailed requirements into list of tasks with as much context as possible that are crucial to follow during completion", required=True)
    project_folder_structure: str = Field(description="Project folder structure to follow.", required=True)
    next_task: str = Field(description="Next Task to do with all the functional and non functional details related to that task", required=True)
    call_next: str = Field(description="name of the node that the flow has to follow next", required=True)
    project_status: str = Field(description="status of the project. whether all the tasks needed for project completion are completed or not", required=True)


    # deliverables: Dict[str, str] = Field(default_factory=dict, description="A seperate Dictionary of Tasks and their corresponding details for completion", required=True)
    description: ClassVar[str] = "Schema of what all documents should be generated."

# Pydantic class used by coder chain for structuring output
class CoderModel(BaseModel):
    """Coder agent output"""

    steps_to_complete: str = Field(description="If the task cannot be completed in one step and needs external tool", required=True)
    files_to_create: str = Field(description="What all files needs to be created", required=True)
    file_path: str = Field(description="Depending on the project structure where should the code be written to", required=True)
    code: str = Field(description="Fully complete, well documented code, with all the naming standards followed that is needed to complete the task.", required=True)
    license_text: str = Field(description="A Licensing text from the user input that needs to be prefixed to each code.", required=True)
    project_status: str = Field(description="status of the project. whether all the tasks needed for project completion are completed or not", required=True)
    call_next: str = Field(description="name of the node that the flow has to follow next", required=True)

    # deliverables: Dict[str, str] = Field(default_factory=dict, description="A seperate Dictionary of Tasks and their corresponding details for completion")
    description: ClassVar[str] = "Schema of task completion output from Coder."


## Pydantic classes for validation

In [ ]:
class ToolChainValidator(BaseModel):
    content: Union[str, List[Union[str, Dict]]]
    tool_calls: list[ToolCall] = Field(min_length=1)
    additional_kwargs: dict


In [ ]:
try:
    ToolChainValidator(content=["hjkl"], tool_calls=[], additional_kwargs={})
except ValidationError as e:
    print(e)

## Prompts

In [ ]:
architect_prompt = ChatPromptTemplate.from_messages(
    [
        (
            ChatRoles.SYSTEM.value,
    
            """<instructions> You are a Development Lead in charge of implementing the given project. Thoroughly analyze the user input and build a thorough requirements document needed to implement the project. 
            You should also be able to break them into independent tasks that can be assigned to other team memeber.\
            Enforce the use of microservice architecture, Best practices Project Folder structure, 12-factor application standards,\
            domain-driven microservice application design, clean-code development architecture standards in the requirements document\
            Final project should include all the source files, configuration files, unit test files, OpenAPI specfile for the project in YAML, License.md file from the User provided URL, a Requirements.txt file, Dockerfile, gitignore and a dockerignore file.
            Structure your answer: 
            1) pick the project name from the user input, 
            2) A well defined complete requirements document, 
            3) Breaking down the tasks into a separate List of tasks that are required to be completed in the project with all the functional and nonfunctional requirements is quintessential and is needed to perform the task smoothly, a list of string should look like ['task1','task2,...,'taskn'].
            4) Project_folder_structure to be enforced for the project 
            5) What should be the next task if we decide to start working on the project. Pick it up sequentially from the available tasks and add any necessary information for the team member to complete the task.\n 
            6) Sequentially assign each task from the tasks field to the next_task field. After each assignment, set the project status to `{pending}`. Once a team member completes a task, confirm its completion before proceeding to the next task. Continue this process until all tasks are completed. Only when all tasks are finished and confirmed by the team members, should the project status be updated to `{completed}`.
            7) Add this to the list of tasks 'write an well defined requirements documentation file in a markdown format to local file system as a 'requirements.md' file in 'docs' folder'.
            Invoke the RequirementsDoc tool to structure the output correctly. </instructions> \n Here is the user question:"""
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
architect_prompt = architect_prompt.partial(pending=TaskState.PENDING.value)
architect_prompt = architect_prompt.partial(completed=TaskState.COMPLETED.value)


coder_prompt = ChatPromptTemplate.from_messages(
    [
        (
            ChatRoles.SYSTEM.value,
    
            """<instructions>
            You are an expert programmer collaborating with the Architect in your team to complete an end to end Coding Project.
            You are good at writing well documented, optimized, secure and productionizable code.
            Here are the standards that you need to follow explicitly for this project:
            1. You do not assume anything and asks Architect for additional context and clarification if requirements are not clear.
            2. Must follow Project Folder Structure decided by Architect.
            3. Must Write the files to the local filesystem.
            4. Follow microservices development standards like 12-factor application standards, domain-driven microservice architecture and clean-code development architecture standards.

            Structure your answer: 
            1) Multiple steps may be needed to complete this task that needs access to some external tools `{coder_tools}`, if so add these steps and mark the project_status as InComplete and call_next to call_tool.
            2) Depending on the project structure where should the code be written to, 
            3) Fully complete, well documented code, with all the naming standards to follow, that is needed to complete the task., 
            4) A Licensing text from the user input that needs to be prefixed to each code. 
            5) Mark the assigned task as '{completed}' and set the call_next to '{architect}', only after receiving the confirmation from one of the external tools. 
            6) Mark the assigned task as '{incomplete}' and set the call_next to '{call_tool}'\n
            Invoke the CoderModel tool to structure the output correctly. </instructions> \n Here is the Architect task:"""
        ),
        MessagesPlaceholder(variable_name="coder_tools"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
coder_prompt = coder_prompt.partial(completed=TaskState.COMPLETED.value)
coder_prompt = coder_prompt.partial(incomplete=TaskState.INCOMPLETE.value)
coder_prompt = coder_prompt.partial(architect=GraphNodes.ARCHITECT.value)
coder_prompt = coder_prompt.partial(call_tool=GraphNodes.CALL_TOOL.value)

tool_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="messages"),
])



## Chains

In [ ]:
architect_chain = architect_prompt | llm.with_structured_output(RequirementsDoc, include_raw=True)

coder_chain = coder_prompt | code_llm.with_structured_output(CoderModel, include_raw=True)

tool_chain = tool_prompt | llm.bind_tools(coder_tools_list)

### Tool executors

In [ ]:
coder_tools_executor = ToolExecutor(coder_tools_list)

## Graph Nodes and their routers

In [ ]:
def architect_node(state: GraphState) -> GraphState:
    """
    architect node
    """
    expected_keys = []
    architect_solution = {}

    # reset the error 
    if state['error']:
       state = toggle_error(state)
    
    if state['project_status'] == TaskState.NEW.value:
        architect_solution = architect_chain.invoke({'messages': get_messages_for_prompt(state)})
        expected_keys = [item for item in RequirementsDoc.__annotations__ if item != "description"]
    elif state['curr_task_status'] == TaskState.AWAITING.value:
        state = add_message(state, (
            ChatRoles.USER.value,
            "Looks like team member has completed the previously assigned task. Please assign new task if any."
        ))
        # temp_state = state.copy()
        # # Deleting 'messages' from 'temp_state' to optimize prompt token size. 
        # del temp_state['messages']

        # architect_solution = architect_chain.invoke({'context': f'{temp_state}', 'messages': get_messages_for_prompt(state)})
        # expected_keys = ['next_task']
    
    # Check if any fields in the schema are missing values.
    missing_keys = [] 
    for key in expected_keys:
        if key not in architect_solution['parsed']:
            missing_keys.append(key)

    if ('parsing_error' in architect_solution) and architect_solution['parsing_error']:
        raw_output = architect_solution['raw']
        error = architect_solution['parsing_error']

        state = toggle_error(state)
        state = add_message(state, (
            ChatRoles.USER.value,
            f"ERROR: parsing your output! Be sure to invoke the tool. Output: {raw_output}. \n Parse error: {error}"
        ))
    elif missing_keys:
        state = toggle_error(state)
        state = add_message(state, (
            ChatRoles.USER.value,
            f"ERROR: Now, try again. Invoke the RequirementsDoc tool to structure the output with a project_name, well_documented, tasks, project_folder_structure, next_task and call_next, you missed {missing_keys} in your previous response",        
        ))
    elif state['curr_task_status'] == TaskState.AWAITING.value:
        state['current_task'] = next_major_task(state)
        state['curr_task_status'] = TaskState.NEW.value

        # print("Architect Task Completion")
        # pp.pp(architect_solution)
        
        if state['current_task'] is None:
            state['project_status'] = TaskState.COMPLETED.value
        else:
            state['project_status'] = TaskState.INCOMPLETE.value
            
        state = add_message(state, (
            ChatRoles.AI.value,
            f"A new task: '{state['current_task']}' has been assigned! Please check the details and start working on it."
        ))
        # TODO->DONE: Use this case for updating the next_step.
        # invoke architect node in such a way that it only updates the next step.
        # maybe adding a new field to state or special value to the task_state 
        # might be helpful for this situation.
        # add message as well stating a new task has been assigned for the coder.
        pass
    else:
        state['requirements_overview'] = architect_solution['parsed']['well_documented']
        state['project_name'] = architect_solution['parsed']['project_name']
        state['project_folder_structure'] = architect_solution['parsed']['project_folder_structure']
        state['tasks'] = ast.literal_eval(architect_solution['parsed']['tasks'])
        state['current_task'] = next_major_task(state)
        state['curr_task_status'] = TaskState.NEW.value

        state = add_message(state, (
            ChatRoles.AI.value,
            "Well document requirements is present now. see what task needs to be completed and do that now.",
        ))

    return state

def architect_router(state: GraphState) -> str:
    """
    architect router
    """
                       #happens when new task need to be assigned
    if state['error'] or (state['curr_task_status'] == TaskState.AWAITING.value):
        return GraphNodes.ARCHITECT.value
    
    if state['project_status'] == TaskState.COMPLETED.value:
        return GraphNodes.END.value

    # if current_task_status is not TaskState.DONE.value then coder
    return GraphNodes.CODER.value

def coder_node(state: GraphState) -> GraphState:
    """
    coder node
    """
    expected_keys = []

    # reset error
    if state['error']:
        state = toggle_error(state)

        
    question = f"Here is a task for you to complete {state['current_task']}."
    context = state['project_folder_structure']

    if state['curr_task_status'] == TaskState.NEW.value:
        coder_solution = coder_chain.invoke({"context": context, "coder_tools": coder_tools, "messages": [(ChatRoles.USER.value, question)]})
        expected_keys = [item for item in CoderModel.__annotations__ if item != "description"]

        if state['project_status'] == TaskState.NEW.value:
            state['project_status'] = TaskState.PENDING.value

        missing_keys = []
        for key in expected_keys:
            if key not in coder_solution['parsed']:
                missing_keys.append(key)
        
        if coder_solution['parsing_error']:
            raw_output = coder_solution['raw']
            error = coder_solution['parsing_error']

            state = toggle_error(state)
            state = add_message(state, (
                ChatRoles.USER.value,
                f"ERROR: parsing your output! Be sure to invoke the tool. Output: {raw_output}. \n Parse error: {error}"
            ))
        elif missing_keys:
            state = toggle_error(state)
            state = add_message(state, (
                ChatRoles.USER.value,
                f"Now, try again. Invoke the CoderModel tool to structure the output with a steps_to_complete, files_to_create, file_path, code, license_text, project_status and call_next. you missed {missing_keys} in your previous response"
            ))
        else:
            state['coder_response'] = coder_solution['parsed']
            state['file_path'] = coder_solution['parsed']['file_path']
            state['files_to_create'] = coder_solution['parsed']['files_to_create']
            state['code'] = coder_solution['parsed']['code']
            state['license_text'] = coder_solution['parsed']['license_text']
            state['project_status'] = coder_solution['parsed']['project_status']
            state['call_next'] = coder_solution['parsed']['call_next']
            coder_steps = re.split(r'\d+\.\s',state['coder_response']['steps_to_complete'])
            state['coder_steps'] = [coder_step.strip() for coder_step in coder_steps if coder_step!='']
            state['curr_task_status'] = TaskState.PENDING.value

    if (len(state['pending_tool_calls']) > 0):
        state = set_curr_tool_call_from_pending_tool_calls(state)
        curr_tool_call = get_curr_tool_call(state)
        state = add_message(state, 
            AIMessage(
                content=f"make a tool_call to '{curr_tool_call['name']}'.",
                additional_kwargs={
                    'tool_calls': [curr_tool_call],
                },
                tool_calls=[curr_tool_call],
                response_metadata={'finish_reason': 'tool_calls'},
            )
        )
    elif (get_next(state) == GraphNodes.CALL_TOOL.value) and (len(state['coder_steps']) > 0):
        state['current_step'] = next_coder_step(state)

        state = add_message(state, (
            ChatRoles.USER.value,
            state['current_step']
        ))

        temp_state = state.copy()
        del temp_state['messages']

        tool_selector = tool_chain.invoke({'context': f'{temp_state}', 'messages': get_messages_for_prompt(state)})

        print("-----Tool Selector-------")
        pp.pp(tool_selector)
        print("-----Tool Selector-------")
        try:
            ToolChainValidator(content=tool_selector.content, tool_calls=tool_selector.tool_calls, additional_kwargs=tool_selector.additional_kwargs)

            state['pending_tool_calls'] += tool_selector.tool_calls
            state = set_curr_tool_call_from_pending_tool_calls(state)
            curr_tool_call = get_curr_tool_call(state)

            if curr_tool_call != {}:
                state = add_message(state, 
                    AIMessage(
                        content=f"make a tool_call to '{curr_tool_call['name']}'.",
                        additional_kwargs={
                            'tool_calls': [curr_tool_call],
                        },
                        tool_calls=[curr_tool_call],
                        response_metadata={'finish_reason': 'tool_calls'},
                    )
                )
            else:
                print("something is wrong! ", curr_tool_call)

            # (
            #         ChatRoles.AI.value,
            #         f"make a tool_call to '{curr_tool_call['name']}' with args '{curr_tool_call['args']}'."
            #     )
            if state['iterations'] > 0:
                state['iterations'] = 0
        except Exception:
                
            if state['iterations'] >= state['max_retry']:
                state['iterations'] = 0

                state = add_message(state, (
                    ChatRoles.USER.value,
                    "MaxRetriesError: Max Retries limit reached. Couldn't finish the step: "
                    f" `{state['current_step']}`."
                ))
            else:
                state = toggle_error(state)
                state['iterations'] += 1
                state['coder_steps'].insert(0, state['current_step'])
                state['pending_tool_calls'] = []
                
                state = add_message(state, (
                    ChatRoles.USER.value,
                    "UnexpectedScenarioOccured: It was unclear whether the tool chain couldn't"
                    " find a suitable tool to complete the task or produced an unintended output"
                    ". An AIMessage with tool calls was expected to be added to the `tool_calls` field in AIMessage" 
                    f"Received: '{tool_selector}'. task:'{state['current_step']}'."
                ))

        print("DEAD END!!!")
        # if tool_selector.tool_calls:
        #     if len(tool_selector.tool_calls) > 0:
        #         state['pending_tool_calls'] += tool_selector.tool_calls
            
        #     state = set_curr_tool_call_from_pending_tool_calls(state)
        #     curr_tool_call = get_curr_tool_call(state)
        #     if curr_tool_call != {}:
        #         state = add_message(state, (
        #             ChatRoles.AI.value,
        #             f"make a tool_call to '{curr_tool_call['name']}' with args '{curr_tool_call['args']}'."
        #         ))
        # else: # Do I really need to reset the whole coder_steps after this?
        #     # TODO->DONE(error, curr_task_status->New): NEED TO UPDATE few flags accordingly to detect this.
        #     # TODO: This case might lead to a infinte loop.
        #     # Think of the case where similiar task as the previous one is given to the tool_selector
        #     # We will end up in this `else` case. even if the task is re structured we will end up in this case 
        #     # again and again.

        #     # state = toggle_error(state)
        #     # state['curr_task_status'] = TaskState.NEW.value

        #     state = add_message(state, (
        #         ChatRoles.USER.value,
        #         f"ERROR: '{tool_selector}' cannot be used to complete '{state['current_step']}' in the task, give an alternative or better breakdown of the task."
        #     ))
    else: # TODO: What if no tools were assigned? control directly came here?
        state = add_message(state, (
            ChatRoles.AI.value,
            f"The task: `{state['current_task']}` was successfully completed. All the steps:"
            f" `{state['coder_response']['steps_to_complete']}` of the task have been addressed."
            f" The task status: `{TaskState.DONE.value}`, Are there any more tasks?"
        ))
        state['curr_task_status'] = TaskState.AWAITING.value
        state['call_next'] = ""
        state['current_step'] = ""
        state['current_task'] = ""
        state['pending_tool_calls'] = []
        state['coder_steps'] = []

    return state

def coder_router(state: GraphState) -> str:
    """
    coder router
    """
    if (get_next(state) == GraphNodes.CALL_TOOL.value) and get_curr_tool_call(state) != {}:
        tool_name: str = get_curr_tool_call(state)['name']
        return tool_name.upper()
    
    if state['error'] or len(state['pending_tool_calls']) > 0 or len(state['coder_steps']) > 0:
        return GraphNodes.CODER.value
    
    # TODO: need to add check for new task assignment
    if (state['project_status'] == TaskState.COMPLETED.value) or (state['curr_task_status'] == TaskState.AWAITING.value):
        return GraphNodes.ARCHITECT.value
    
    return GraphNodes.CODER.value

def download_license_file_node(state: GraphState) -> GraphState:
    """
    download license file node 
    """
    
    tool_call = get_curr_tool_call(state)

    tool_name = tool_call['name']
    tool_args = tool_call['args']
    tool_call_id = tool_call['id']

    action = ToolInvocation(
        tool=tool_name,
        tool_input=tool_args,
    )
    
    response = coder_tools_executor.invoke(action)
    state = add_message(state, 
        ToolMessage(
            content=f"{response}",
            name=tool_name,
            tool_call_id=tool_call_id
        )
    )

    state['curr_tool_call'] = {}
    state['current_step'] = ""

    return state

def write_generated_code_to_file_node(state: GraphState) -> GraphState:
    """
    write generated code to file node
    """

    tool_call = get_curr_tool_call(state)

    tool_name = tool_call['name']
    tool_args = tool_call['args']
    tool_call_id = tool_call['id']

    action = ToolInvocation(
        tool=tool_name,
        tool_input=tool_args,
    )
    
    response = coder_tools_executor.invoke(action)
    # state = add_message(state, (ChatRoles.USER.value, f"tool call response: {response}"))
    state = add_message(state, 
        ToolMessage(
            content=f"{response}",
            name=tool_name,
            tool_call_id=tool_call_id
        )
    )

    state['curr_tool_call'] = {}
    state['current_step'] = ""

    return state

def create_git_repo_node(state: GraphState) -> GraphState:
    """
    create git repo node
    """

    tool_call = get_curr_tool_call(state)

    tool_name = tool_call['name']
    tool_args = tool_call['args']
    tool_call_id = tool_call['id']

    action = ToolInvocation(
        tool=tool_name,
        tool_input=tool_args,
    )
    
    response = coder_tools_executor.invoke(action)
    state = add_message(state, 
        ToolMessage(
            content=f"{response}",
            name=tool_name,
            tool_call_id=tool_call_id
        )
    )

    state['curr_tool_call'] = {}
    state['current_step'] = ""

    return state

def execute_command_node(state: GraphState) -> GraphState:
    """
    execute command node
    """

    tool_call = get_curr_tool_call(state)

    tool_name = tool_call['name']
    tool_args = tool_call['args']
    tool_call_id = tool_call['id']
    
    action = ToolInvocation(
        tool=tool_name,
        tool_input=tool_args,
    )
    
    response = coder_tools_executor.invoke(action)
    state = add_message(state, 
        ToolMessage(
            content=f"{response}",
            name=tool_name,
            tool_call_id=tool_call_id
        )
    )

    state['curr_tool_call'] = {}
    state['current_step'] = ""

    return state

## Need for Persistence
In LangGraph, memory for maintaining context across interactions is facilitated via Checkpointers within StateGraphs.  
1. When setting up a LangGraph workflow, you can ensure state persistence by employing a Checkpointer like `AsyncSqliteSaver`.  
2. Simply include this in your workflow setup by calling `compile(checkpointer=my_checkpointer)` during graph compilation.

In [ ]:
memory = SqliteSaver.from_conn_string(":memory:")

In [ ]:
# variable setup
architect_str = GraphNodes.ARCHITECT.value
coder_str = GraphNodes.CODER.value
end_str = GraphNodes.END.value

dlf_str = GraphNodes.DOWNLOAD_LICENSE_FILE.value
wgcf_str = GraphNodes.WRITE_GENERATED_CODE_TO_FILE.value
cgr_str = GraphNodes.CREATE_GIT_REPO.value
ec_str = GraphNodes.EXECUTE_COMMAND.value

workflow = StateGraph(GraphState)

# add all the node requied for the graph
workflow.add_node(architect_str, architect_node)
workflow.add_node(coder_str, coder_node)
workflow.add_node(dlf_str, download_license_file_node)
workflow.add_node(wgcf_str, write_generated_code_to_file_node)
workflow.add_node(cgr_str, create_git_repo_node)
workflow.add_node(ec_str, execute_command_node)

# add edges for the graph
workflow.add_conditional_edges(architect_str, architect_router, {
    architect_str: architect_str,
    coder_str: coder_str,
    end_str: end_str
})

workflow.add_conditional_edges(coder_str, coder_router, {
    architect_str: architect_str,
    coder_str: coder_str,
    dlf_str: dlf_str,
    wgcf_str: wgcf_str,
    cgr_str: cgr_str,
    ec_str: ec_str
})

workflow.add_edge(wgcf_str, coder_str)
workflow.add_edge(dlf_str, coder_str)
workflow.add_edge(cgr_str, coder_str)
workflow.add_edge(ec_str, coder_str)

workflow.set_entry_point(architect_str)

workflow_graph = workflow.compile(checkpointer=memory)

## Workflow call graph

In [ ]:
try:
    display(Image(workflow_graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(e)
    # This requires some extra dependencies and is optional
    pass

# Inovke graph

In [2]:
user_input, license_text = read_input_json("rest_api.json")

new_state = {
    'error': False,
    'generation': "",
    'iterations': 0,
    'curr_tool_call': {},
    'curr_task_status': TaskState.NEW.value,
    'current_step': "",

    'project_name': "",
    'requirements_overview': "",
    'tasks': [],
    'project_folder_structure': "",
    'current_task': "",

    'file_path': "",
    'code': "",
    'coder_steps': [],
    'coder_response': "",
    'pending_tool_calls': [],
    'license_text': "",
    
    'project_status': TaskState.NEW.value,
    'call_next': GraphNodes.NONE.value,
    'max_retry': 2,
}
events = workflow_graph.stream(
    {   
        **new_state,
        "messages": [
            (   "user",
                f"Create this project for me in {PROJECT_PATH}." 
                f"Requirements are {user_input}."
                f"{license_text} must be present at the top of each file created as part of the project." 
                "Once you code it up, finish."
            )
        ]
    },
    # Maximum number of steps to take in the graph and the thread ID to be used to persist the memory.
    {
        "recursion_limit": 200,
        "configurable": {"thread_id": "1"}
    },
)

for s in events:
    pp.pp(s)
    print("----")

NameError: name 'read_input_json' is not defined